In [98]:
# Cuts used to go from o.summary.fits to o.gst.fits, for now we will use the o.gst.fits file

#snr = 5.0
#sharp = 0.04
#crowd = 0.5
#objtype = 1
#flag = 99

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import numpy as np


In [141]:
def writeregionfile(filename, objlist, color="blue",sys=''):
    if sys == '': sys = 'wcs'
    out = open(filename,'w')
    i = -1
    out.write('# Region file format: DS9 version 4.0\nglobal color='+color+' font="helvetica 10 normal" select=1 highlite=1 edit=1 move=1 delete=1 include=1 fixed=0 source\n')
    if sys == 'wcs':
      out.write('fk5\n')
      for i in range(len(objlist)):
        out.write("point(%.7f,%.7f) # point=boxcircle text={%i}\n" % (objlist.RA[i],objlist.DEC[i], i))
    if sys == 'img':
      out.write('image\n')
      for i in range(len(objlist)):
        out.write("point(%.3f,%.3f) # point=boxcircle text={%i}\n" % (objlist.X[i],objlist.Y[i], i))
    out.close()
    print("writing region file:", filename)


In [142]:
sn_position = SkyCoord(ra=309.33000691667*u.degree, dec=+66.106417016667*u.degree)
sn_name='2021sjt'
filt='F555W'
mjd='56789'
search_radius = 3 * u.arcsec
path='test_photometry/'

In [143]:
# Read in the photometry for the filtered catalog and the summary catalog
cut_catalog=fits.open(path+'o.gst.fits')[1]
full_catalog=fits.open(path+'o.summary.fits')[1]

data_cut=cut_catalog.data
data_full=full_catalog.data

tbl_cut=Table(data_cut)
tbl_full=Table(data_full)

radec_cut = SkyCoord(ra=data_cut.RA*u.degree, dec=data_cut.DEC*u.degree)
radec_full = SkyCoord(ra=data_full.RA*u.degree, dec=data_full.DEC*u.degree)

In [144]:
d2d_cut = sn_position.separation(radec_cut)
d2d_full = sn_position.separation(radec_full)

index_cut=d2d_cut < search_radius
index_full=d2d_full < search_radius

data_searched_cut=data_cut[index_cut]
data_searched_full=data_full[index_full]

tbl_searched_cut=tbl_cut[index_cut]
tbl_searched_full=tbl_full[index_full]


In [145]:
writeregionfile(sn_name+"-"+filt+"-"+mjd+str(search_radius) +"_ra_dec.reg", data_searched_cut, "red", "wcs")
writeregionfile(sn_name+"-"+filt+"-"+mjd+str(search_radius) +"_pixel_xy.reg", data_searched_cut, "red", "img")

writing region file: 2021sjt-F555W-567893.0 arcsec_ra_dec.reg
writing region file: 2021sjt-F555W-567893.0 arcsec_pixel_xy.reg
